In [ ]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from sentence_transformers import SentenceTransformer, util


In [ ]:
s_model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
extended_texts = []
with open("generatedTextTraining.txt", 'r') as file:
  # txt = file.read()
  for line in file:
    if(line == "----------------------------------------------------\n" or line == "\n"):
      continue
    extended_texts.append(line)
extended_texts

In [ ]:
# Replace 'your_file.txt' with the actual ten
shortTextSentences = []
with open('shortText_full.txt', 'r') as f:
  for line in f:
    shortTextSentences.append(line[0:len(line)-1])

# Print the contents of the file
# print(text)
# sentences


In [ ]:
len(shortTextSentences)

30902

In [ ]:
def appendThre(threshold_value, sorted_dict, text):
  score_sent1 = {}
  sentences2 = []
  ct = 0
  temp_text = text

  n_sentences = 0

  with open('similarity_score_0.4_1.txt', 'a', encoding='utf-8') as fss:
    fss.write("----------------------------------------------------\n")
    for key, value in sorted_dict.items():

      if key >= threshold_value:
        fss.write(f"{key}: {value}\n")
        temp_text = temp_text + " " + value
        n_sentences += 1
        ct += 1
      elif ct == 0:
        return sorted_dict, text, -1, n_sentences
      else:
        # fss.write(f"{n_sentences}\n")

        sentences2.append(value)
    fss.write(f"{n_sentences}\n")

  sentences1 = []

  for i in range(len(sentences2)):
    sentences1.append(temp_text)


  embeddings1 = s_model.encode(sentences1, convert_to_tensor=True)
  embeddings2 = s_model.encode(sentences2, convert_to_tensor=True)

  cosine_scores = util.cos_sim(embeddings1, embeddings2)

  for i in range(len(sentences2)):
    score_sent1[cosine_scores[i][i]] = sentences2[i]

  # thresh_value = 0.7

  score_sent = dict(sorted(score_sent1.items(), key=lambda item: item[0], reverse=True))

  if len(score_sent) == 0:
    return score_sent, temp_text, -1, n_sentences

  return score_sent, temp_text, threshold_value, n_sentences

In [ ]:
extended_sentences = []
ct=0
ind=0
total_no_of_sentences = 0


for text in shortTextSentences[0:1000]:
    print(ct)
    ct+=1

    no_of_sentence_add_per_text = 0

    try:

        gener_text = extended_texts[ind]
        ind+=1
        # new_text = tokenizer.decode(gener_text, skip_special_tokens=True)
        sentences = nltk.sent_tokenize(gener_text)


        # Tokenize sentences
        sentences1 = [text] * len(sentences)
        sentences2 = sentences

        embeddings1 = s_model.encode(sentences1, convert_to_tensor=True)
        embeddings2 = s_model.encode(sentences2, convert_to_tensor=True)
        cosine_scores = util.cos_sim(embeddings1, embeddings2)
        sumOfScores = 0
        iteration_num = 0
        score_sent = {}
        for i in range(len(sentences1)):
            sumOfScores = sumOfScores + cosine_scores[i][i]
            score_sent[cosine_scores[i][i]] = sentences2[i]

        threshold_value = 0.4

        sorted_dict = dict(sorted(score_sent.items(), key=lambda item: item[0], reverse=True))
        print(len(sorted_dict))

        with open('similarity_score_0.4_1.txt', 'a', encoding='utf-8') as fss:

          for key, value in sorted_dict.items():
            fss.write(f"{key} : {value}\n")
          fss.write("----------------------------------------------------\n")

        final_text = text

        with open('similarity_score_0.4_1.txt', 'a', encoding='utf-8') as fss:

          while True:
            score_sent1, text1, thresh_value, no_of_sentences = appendThre(threshold_value, sorted_dict, final_text)
            # fss.write(f"{no_of_sentences}\n")
            total_no_of_sentences += no_of_sentences
            no_of_sentence_add_per_text += no_of_sentences
            if thresh_value == -1:
              break

            sorted_dict = score_sent1
            final_text = text1





        extended_sentences.append(final_text)
        with open('similarity_score_0.4_1.txt', 'a', encoding='utf-8') as fss:
          fss.write(f"{no_of_sentence_add_per_text}\n")
          fss.write("====================================================\n")

        with open('threshold_0.4_1.txt', 'a', encoding='utf-8') as f1:
        # Write your data to the file
            f1.write(final_text)
            f1.write("\n----------------------------------------------------\n")
    except IndexError as e:
        # Handle the IndexError here
        print("An IndexError occurred:", e)
        # Append the original text and continue
        extended_sentences.append(text+"-1")
        continue

    print("___________________")

# extended_sentences
print(total_no_of_sentences)

0
22
___________________
1
22
___________________
2
12
___________________
3
20
___________________
4
18
___________________
5
17
___________________
6
15
___________________
7
23
___________________
8
15
___________________
9
20
___________________
10
22
___________________
11
24
___________________
12
17
___________________
13
3
___________________
14
24
___________________
15
27
___________________
16
15
___________________
17
29
___________________
18
20
___________________
19
15
___________________
20
21
___________________
21
22
___________________
22
23
___________________
23
18
___________________
24
21
___________________
25
18
___________________
26
15
___________________
27
30
___________________
28
24
___________________
29
44
___________________
30
20
___________________
31
19
___________________
32
18
___________________
33
19
___________________
34
23
___________________
35
18
___________________
36
18
___________________
37
33
___________________
38
12
_________________

In [ ]:
T